# Processing the final dataset



In [1]:
import ee
import geemap

# Authenticate to Earth Engine
try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()

data_folder = "projects/amazon-forest-regrowth/assets"

In [ ]:
biomass_all = ee.Image(f"{data_folder}/raw/biomass")
proj = biomass_all.projection().getInfo()

mature_biomass = ee.Image(f"{data_folder}/mature_biomass_1km").reproject(
    crs = proj["crs"], crsTransform = proj["transform"]
)

# scale = biomass_all.projection().nominalScale().getInfo()

task = ee.batch.Export.image.toDrive(
    image = mature_biomass_1k,
    description = 'mature_biomass_1k',
    region = mature_biomass_1k.geometry(),
    scale = 500,
    maxPixels = 1e13
)
task.start()


In [11]:
years = 0

if (years == 0):
    land_use_years = ee.Image(f"{data_folder}/land_use")
    name = "all_LULC_dist_amaz_500"
else:
    land_use_years = ee.Image(f"{data_folder}/land_use_{years}_years")
    name = f"{years}y_LULC_dist_amaz_500"

age_agbd = ee.Image(f"{data_folder}/age_agbd")
categorical = ee.Image(f"{data_folder}/categorical")
sur_cover = ee.Image(f"{data_folder}/sur_cover").rename("sur_cover")
one_hectare_mask = ee.Image(f"{data_folder}/one_hectare_mask").rename("one_hectare_mask")
cwd = ee.Image(f"{data_folder}/raw/cwd_chave").float().rename("cwd")
yearly_mean_prec = ee.Image(f"{data_folder}/yearly_mean_prec")
yearly_SI = ee.Image(f"{data_folder}/yearly_SI")
distance = ee.Image(f"{data_folder}/distance_amaz_500")

combined_mask = one_hectare_mask.And(land_use_years.select("last_LU")).And(distance)

unified_data = (
    age_agbd.addBands(
        [categorical, land_use_years, cwd, distance, sur_cover, yearly_mean_prec, yearly_SI
    ]).updateMask(combined_mask)
)

unified_data_fc = unified_data.stratifiedSample(
    numPoints = 5000, classBand = 'ecoreg', region = unified_data.geometry()
)

# Export an ee.FeatureCollection to Google Drive, adjusting the description based on the years
task = ee.batch.Export.table.toDrive(
    collection=unified_data_fc, description=name
)
task.start()
